In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import logging

In [10]:
train_X = torch.load('../../../../derived_data/sample-pr_psl/train_X.pt')
train_y = torch.load('../../../../derived_data/sample-pr_psl/train_y.pt')

val_X = torch.load('../../../../derived_data/sample-pr_psl/val_X.pt')
val_y = torch.load('../../../../derived_data/sample-pr_psl/val_y.pt')

In [11]:
print(train_X.shape)
print(train_y.shape)
print(val_X.shape)
print(val_y.shape)

torch.Size([100, 2, 53, 53])
torch.Size([100, 1, 53, 53])
torch.Size([100, 2, 53, 53])
torch.Size([100, 1, 53, 53])


In [14]:
train_dl = DataLoader(TensorDataset(train_X, train_y), batch_size=64)
val_dl = DataLoader(TensorDataset(val_X, val_y), batch_size=64)

In [27]:
def train_on_batch(batch_X, batch_y, model):
    # Compute prediction and loss
    outputs_tensor = model(batch_X)
    loss = criterion(outputs_tensor, batch_y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss

def train_epoch(model, dataloader, device, epoch):
    model.train()

    epoch_loss = 0.0

    for i, (batch_X, batch_y) in enumerate(dataloader):
        loss = train_on_batch(batch_X.to(device), batch_y.to(device), model)
        epoch_loss += loss.item()

        # Log progress at least every 10th batch
        if (len(dataloader) <= 10) or ((i+1) % max(len(dataloader)//10,1) == 0):
            logging.info(f"Epoch {epoch}: Batch {i}: Batch Train Loss {loss.item()} Running Epoch Train Loss {epoch_loss}")

    return epoch_loss

In [99]:
device = 'cpu'
model = nn.Conv2d(2, 1, kernel_size=31, padding=15)
learning_rate = 2e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss().to(device)

train_epoch(model, train_dl, device, 0)

329252.625

In [95]:
model(list(enumerate(train_dl))[1][1][0]).shape

torch.Size([36, 1, 55, 55])